실습 #1: gradio의 chatbot interface를 사용하여 ChatGPT와 스트리밍하면서 대화하는 인터페이스를 구성하시오.
실습 #2: 만든 어플리케이션을 멀티턴으로 수정하시오.

In [2]:
!pip install -U gradio

  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 17.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 16.0 MB/s eta 0:00:00a 0:00:01


In [30]:
import os
import gradio as gr
import random
import time
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def user(message, chat_history):
        chat_history.append((message, ""))
        return "", chat_history
    
    def respond(message, chat_history):
        messages = [{"role": "system", "content": "You are a helpful assistant."}]
        for user_msg, bot_msg in chat_history:
            messages.append({"role": "user", "content": user_msg})
            if bot_msg:
                messages.append({"role": "assistant", "content": bot_msg})
        
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            stream=True,
        )
        
        full_response = ""
        for chunk in response:
            delta = chunk.choices[0].delta
            if hasattr(delta, "content") and delta.content:
                full_response += delta.content

        chat_history[-1] = (chat_history[-1][0], full_response)
        return "", chat_history

    clear.click(lambda: None, None, chatbot, queue=False)
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    # msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
    #     respond, chatbot, chatbot
    # )

demo.launch(server_name='0.0.0.0')

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7887

To create a public link, set `share=True` in `launch()`.
